In [ ]:
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
import matplotlib.pyplot as plt


In [ ]:
def preprocess_audio(audio_path):
    
    y, sr = librosa.load(audio_path, sr=16000)
    
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    
    log_spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
   
    log_spectrogram -= log_spectrogram.min()
    log_spectrogram /= log_spectrogram.max()
    
    log_spectrogram = np.expand_dims(log_spectrogram, axis=-1)
    log_spectrogram = np.expand_dims(log_spectrogram, axis=0)  # Add batch dimension
    return log_spectrogram


In [ ]:
def build_resnet_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

input_shape = (128, None, 1)  
model = build_resnet_model(input_shape)


In [ ]:
audio_path = 'audio.wav'
spectrogram = preprocess_audio(audio_path)

prediction = model.predict(spectrogram)
probability = prediction[0][0]
print(f'Deepfake Probability: {probability:.2f}')


In [ ]:
def wavefake_analysis(audio_path):
    
    y, sr = librosa.load(audio_path, sr=16000)
    y = y[:16000]  # Use first second for simplicity
    y = np.expand_dims(y, axis=0)  
    y = np.expand_dims(y, axis=-1)  

    # Predict deepfake probability
    prediction = model.predict(y)
    probability = prediction[0][0]
    return probability

probability_wavefake = wavefake_analysis(audio_path)
print(f'WaveFake Deepfake Probability: {probability_wavefake:.2f}')
